In [ ]:
# Core
import sys
import pandas as pd
import numpy as np
from pathlib import Path

## Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

## Harp/Bonsai
sys.path.append('../../src/')
from bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
import harp
import harp.processing
import data_io



In [ ]:
#Global Viz settings
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

default_img_size = (15, 8)

In [ ]:
session_path = Path(r"Z:\scratch\vr-foraging\672107\20230929T111709")

# Harp Devices:
HarpBehavior = harp.HarpDevice("Behavior")
harp_behavior_data = data_io.HarpSource(device=HarpBehavior, path=session_path / "Behavior", name="behavior", autoload=False)

# Software Events
software_events = data_io.SoftwareEventSource(path=session_path / "SoftwareEvents", name="software_events")


In [ ]:
patches = software_events.streams.ActivePatch.data
reward = software_events.streams.GiveReward.data
reward_available = patches.iloc[0]["data"]["patchRewardFunction"]["initialRewardAmount"]

reward_updates = pd.concat([patches, reward])
reward_updates.sort_index(inplace=True)
reward_updates["currentReward"] = np.nan
for event in reward_updates.iterrows():
    if event[1]["name"] == 'GiveReward': #update reward
        reward_available -= event[1]["data"]
    elif event[1]["name"] == 'ActivePatch': #reset reward
        reward_available = event[1]["data"]["patchRewardFunction"]["initialRewardAmount"]
    else:
        raise ValueError("Unknown event type")
    reward_updates.at[event[0], "currentReward"] = reward_available


In [ ]:
# Align velocity to each first odor onset per reward site

# Get the first odor onset per reward site
sites = software_events.streams.ActiveSite.data
reward_sites = sites[sites["data"].apply(lambda x: x["label"] == "Reward")].copy()

# Odor onsets
## mask for digital outputs
digital_outputs = HarpBehavior.module.DigitalOutputs

## Load necessary files
harp_behavior_data.streams.OutputSet.load_from_file()
harp_behavior_data.streams.OutputClear.load_from_file()
odor_0 = harp.processing.distinct_until_changed_state(harp_behavior_data.streams.OutputSet.data, harp_behavior_data.streams.OutputClear.data, digital_outputs.SupplyPort1)
odor_1 = harp.processing.distinct_until_changed_state(harp_behavior_data.streams.OutputSet.data, harp_behavior_data.streams.OutputClear.data, digital_outputs.SupplyPort2)

odors_onset = pd.concat(
    [
        odor_0[odor_0["Value"]].assign(Value=0),
        odor_1[odor_1["Value"]].assign(Value=1),
    ], axis=0, copy=True).sort_index()


## For each reward, get the first odor onset
reward_sites["OdorOnset.Time"] = np.nan
reward_sites["OdorOnset.OdorIdentity"] = np.nan
reward_sites["RewardAvailable"] = np.nan

for site in reward_sites.itertuples():
    arg_min, val_min = harp.processing.find_closest(site.Index, odors_onset.index, mode="closest")
    if arg_min:
        reward_sites.loc[site.Index, "OdorOnset.Time"] = val_min
        reward_sites.loc[site.Index, "OdorOnset.OdorIdentity"] = odors_onset["Value"].iloc[arg_min]
    arg_min, val_min = harp.processing.find_closest(site.Index, reward_updates.index.values, mode="below_zero")
    reward_sites.loc[site.Index, "RewardAvailable"] = reward_updates["currentReward"].iloc[arg_min]

## Load data from encoder efficiently
encoder_data = harp.read_harp_bin(harp_behavior_data.streams.AnalogData.path)[1]
# Resample encoder_data
encoder_data.index = pd.to_datetime(encoder_data.index, unit="s")
encoder_data = encoder_data.resample("16.6ms").sum().interpolate(method="linear")
encoder_data.index = (encoder_data.index - pd.to_datetime(0))
encoder_data.index = encoder_data.index.total_seconds()
encoder_data = encoder_data * -1

In [ ]:
window = (-0.2, 1)
fig, axs = plt.subplots(1,2)
fig.tight_layout()
fig.set_size_inches(default_img_size)

pallete = sns.color_palette("Spectral", n_colors=7)

ylim = (-50, 200)

t_limit = 100_000

for i, (odor_label, odor_group) in enumerate(reward_sites.groupby("OdorOnset.OdorIdentity")):
    for trial in odor_group.iterrows():
        if trial[0] < t_limit:
            odor_onset = trial[1]["OdorOnset.Time"]
            enconder_slice = encoder_data.loc[odor_onset + window[0]: odor_onset + window[1]]
            color = pallete[int(trial[1]["RewardAvailable"])]
            axs[i].plot(
                enconder_slice.index.values - odor_onset,
                enconder_slice.values,
                color=color, alpha=0.2, lw = 2)
            axs[i].set_ylim(ylim)
            axs[i].vlines(0, *axs[i].get_ylim(), color="k", linestyle="--")
    axs[i].set_title(f"Odor {int(odor_label)}")
    axs[i].set_xlabel("Time(s)")
    axs[i].set_ylabel("Velocity (encoder bits/s)")


In [ ]:
trial